In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("FakeFriendsQueries").getOrCreate()

23/12/14 17:21:08 WARN Utils: Your hostname, perezs-zeenbook resolves to a loopback address: 127.0.1.1; using 192.168.49.205 instead (on interface wlo1)
23/12/14 17:21:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/14 17:21:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# read csv
friends = spark.read.csv(
    "../data/fakefriends.csv",
    inferSchema=True,
)

# rename columns
col_names = ["index", "name", "age", "friends"]
friends = friends.toDF(*col_names)

# we only care about age and friends
friends = friends.select("age", "friends")

friends.show(5)

+---+-------+
|age|friends|
+---+-------+
| 33|    385|
| 26|      2|
| 55|    221|
| 40|    465|
| 68|     21|
+---+-------+
only showing top 5 rows



In [4]:
# query avg friends per age
avg_friends_per_age = friends.groupBy("age").mean("friends")
# round avg column and rename it
avg_friends_per_age = avg_friends_per_age.withColumn(
    "avg_friends", F.round(avg_friends_per_age["avg(friends)"], 1)
)
# sort by age
avg_friends_per_age = avg_friends_per_age.sort("age")
avg_friends_per_age.show(5)

+---+------------------+-----------+
|age|      avg(friends)|avg_friends|
+---+------------------+-----------+
| 18|           343.375|      343.4|
| 19|213.27272727272728|      213.3|
| 20|             165.0|      165.0|
| 21|           350.875|      350.9|
| 22|206.42857142857142|      206.4|
+---+------------------+-----------+
only showing top 5 rows



In [5]:
spark.stop()